In [ ]:
%cd drive/MyDrive/BUP_Video_Classification/

/content/drive/MyDrive/BUP_Video_Classification


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# ==============================================================================
# BUŇKA 1: INICIALIZACE PROSTŘEDÍ (LOKÁLNÍ VERZE)
# (Spusťte vždy jako první po spuštění notebooku)
# ==============================================================================
import os
import sys
from dotenv import load_dotenv

# 1. Nastavení hlavních cest
BASE_PROJECT_PATH = os.getcwd()
print(f"Pracovní složka projektu: {BASE_PROJECT_PATH}")

# 2. Nastavení odvozených cest
SRC_PATH = os.path.join(BASE_PROJECT_PATH, "src")
DATA_PATH = os.path.join(BASE_PROJECT_PATH, "data")
MODEL_INPUTS_PATH = os.path.join(DATA_PATH, "model_inputs")
MODELS_PATH = os.path.join(BASE_PROJECT_PATH, "models") # Nová složka pro .pkl modely
RESULTS_PATH = os.path.join(BASE_PROJECT_PATH, "results") # Nová složka pro .csv s výsledky

# Vytvoříme složky, pokud neexistují
#os.makedirs(MODEL_INPUTS_PATH, exist_ok=True)
#os.makedirs(MODELS_PATH, exist_ok=True)
#os.makedirs(RESULTS_PATH, exist_ok=True)


# 3. Přidání složky 'src' do sys.path (aby fungovaly importy)
if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)
    print(f"Složka 'src' přidána do sys.path")
else:
    print(f"Složka 'src' je již v sys.path")

# 4. Načtení API klíče z .env souboru
load_dotenv()
API_KEY_CHECK = os.getenv('OPENAI_API_KEY')
if API_KEY_CHECK and len(API_KEY_CHECK) > 10:
    print("OPENAI_API_KEY úspěšně načten z .env souboru.")
else:
    print("VAROVÁNÍ: OPENAI_API_KEY nenalezen.")

print("\n--- Prostředí je připraveno ---")

Pracovní složka projektu: /Users/kovy/Library/CloudStorage/GoogleDrive-koval.macek@gmail.com/My Drive/BUP_Video_Classification
Složka 'src' je již v sys.path
OPENAI_API_KEY úspěšně načten z .env souboru.

--- Prostředí je připraveno ---


In [6]:
# ==============================================================================
# BUŇKA 2: GLOBÁLNÍ IMPORTY A NASTAVENÍ
# ==============================================================================
import pandas as pd
import numpy as np
import importlib
import os
import pickle
import openai

# Import našich vlastních modulů
import src.data_preparation as dp
import src.video_processing as vp
import src.llm_generation as lg
import src.feature_engineering as fe
#import src.modeling as md
#import src.plotting as pl
#import src.xai as xai

# Nastavení pro hezčí výpisy
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 150)

print("Všechny moduly naimportovány.")

Všechny moduly naimportovány.


In [7]:
# ==============================================================================
# BUŇKA 3: FÁZE 1 - PŘÍPRAVA DAT
# (Tuto buňku stačí spustit jen jednou)
# ==============================================================================
print("--- FÁZE 1: PŘÍPRAVA DAT ---")
importlib.reload(dp)

selected_classes_list = [
    'ApplyEyeMakeup', 'ApplyLipstick', 'Archery', 'BabyCrawling', 'Basketball',
    'BenchPress', 'Biking', 'Billiards', 'BlowDryHair', 'BrushingTeeth',
    'CleanAndJerk', 'CliffDiving', 'CuttingInKitchen', 'Fencing', 'GolfSwing'
]

train_split_file = os.path.join(DATA_PATH, "train_split.csv")
if os.path.exists(train_split_file):
    print("Soubory 'train_split.csv' a 'test_split.csv' již existují. Přeskakuji Fázi 1.")
else:
    print("Vytvářím mini-dataset...")
    dp.create_mini_dataset(
        base_data_path=DATA_PATH,
        selected_classes=selected_classes_list,
        full_dataset_folder="UCF101_full/UCF-101" # Ujistěte se, že tato cesta je správná
    )

    print("Vytvářím train/test splity...")
    dp.create_group_splits(base_data_path=DATA_PATH)
    print("Fáze 1 dokončena.")

--- FÁZE 1: PŘÍPRAVA DAT ---
Soubory 'train_split.csv' a 'test_split.csv' již existují. Přeskakuji Fázi 1.


In [9]:
# ==============================================================================
# BUŇKA 4: FÁZE 2 - GENEROVÁNÍ POPISŮ (LLM)
# (Tato buňka poběží dlouho a stojí peníze. Má resume logiku.)
# ==============================================================================
print("--- FÁZE 2: GENEROVÁNÍ POPISŮ (LLM) ---")
importlib.reload(lg)
importlib.reload(vp)

lg.generate_all_descriptions(
    data_path=DATA_PATH,
    num_frames_per_video=3, # Začněte s 3 snímky
    rate_limit_pause=0.5
)

print("\n--- Fáze 2 je kompletní ---")

--- FÁZE 2: GENEROVÁNÍ POPISŮ (LLM) ---
OpenAI klient úspěšně vytvořen.
Celkem videí v datasetu: 1986
Zbývá zpracovat: 1986


Generování popisů:   0%|          | 1/1986 [00:50<27:45:30, 50.34s/it]


KeyboardInterrupt: 

In [ ]:
# ==============================================================================
# BUŇKA 5: FÁZE 3 - FEATURE ENGINEERING (VNOŘENÍ A AGREGACE)
# (Tato buňka poběží dlouho a stojí peníze.)
# ==============================================================================
print("--- FÁZE 3: FEATURE ENGINEERING ---")
importlib.reload(fe)

fe.process_and_embed_all(data_path=DATA_PATH)

print("\n--- Fáze 3 je kompletní ---")

--- FÁZE 2 (Start): Generování popisů pomocí LLM ---
Celkem videí ke zpracování: 1986


Generování popisů:   3%|▎         | 52/1986 [32:48<20:19:55, 37.85s/it]


KeyboardInterrupt: 